In [1]:
import requests
import requests
import pandas as pd
from datetime import datetime, timedelta

## Air quality data

In [3]:
class AirQualityData:
    def __init__(self):
        self.api_url = "https://api.data.gov.sg/v1/environment/psi"

    def get_psi_data(self, start_date, end_date):
        """
        Fetch historical PSI data for Singapore from NEA API within a date range.
        
        Args:
            start_date (str): Start date in 'YYYY-MM-DD' format (e.g., '2014-01-01').
            end_date (str): End date in 'YYYY-MM-DD' format (e.g., '2024-01-01').

        Returns:
            pd.DataFrame: DataFrame containing PSI data for all regions between start_date and end_date.
        """
        psi_data = []

        # Convert string dates to datetime objects
        start = datetime.strptime(start_date, '%Y-%m-%d')
        end = datetime.strptime(end_date, '%Y-%m-%d')

        # Loop through each day in the range
        delta = timedelta(days=1)
        while start <= end:
            date_str = start.strftime('%Y-%m-%dT%H:%M:%S')
            try:
                # Make API request for each day
                response = requests.get(self.api_url, params={"date_time": date_str})
                response.raise_for_status()
                data = response.json()
                
                # Extract PSI readings for all regions
                if 'items' in data and data['items']:
                    for region in data['region_metadata']:
                        name = region['name']
                        readings = data['items'][0]['readings']
                        psi_entry = {
                            "region": name,
                            "date": start.strftime('%Y-%m-%d'),
                            "psi_24hr": readings['psi_twenty_four_hourly'][name],
                            "pm10": readings['pm10_twenty_four_hourly'][name],
                            "pm25": readings['pm25_twenty_four_hourly'][name],
                            "so2": readings['so2_twenty_four_hourly'][name],
                            "co": readings['co_eight_hour_max'][name],
                            "o3": readings['o3_eight_hour_max'][name]
                        }
                        psi_data.append(psi_entry)
            
            except requests.exceptions.RequestException as e:
                print(f"Error fetching PSI data for {date_str}: {e}")
            
            # Move to the next day
            start += delta

        # Convert list of dictionaries to DataFrame
        psi_df = pd.DataFrame(psi_data)
        return psi_df

# Example usage:
air_quality = AirQualityData()
psi_df = air_quality.get_psi_data("2016-01-01", "2024-01-01")
print(psi_df.head())

Error fetching PSI data for 2016-01-01T00:00:00: 500 Server Error: Internal Server Error for url: https://api.data.gov.sg/v1/environment/psi?date_time=2016-01-01T00%3A00%3A00
Error fetching PSI data for 2016-01-02T00:00:00: 500 Server Error: Internal Server Error for url: https://api.data.gov.sg/v1/environment/psi?date_time=2016-01-02T00%3A00%3A00
     region        date  psi_24hr  pm10  pm25   so2    co    o3
0  national  2016-03-10      52.0  26.0  13.0  14.0  0.60  39.0
1     south  2016-03-10      52.0  26.0  13.0   7.0  0.31  31.0
2     north  2016-03-10      50.0  25.0  12.0   3.0  0.44  39.0
3      east  2016-03-10      34.0  15.0   8.0   5.0  0.60  38.0
4   central  2016-03-10      43.0  20.0  10.0   7.0  0.35  34.0


In [4]:
psi_df

,region,date,psi_24hr,pm10,pm25,so2,co,o3
0,national,2016-03-10,52.0,26.0,13.0,14.0,0.60,39.0
1,south,2016-03-10,52.0,26.0,13.0,7.0,0.31,31.0
2,north,2016-03-10,50.0,25.0,12.0,3.0,0.44,39.0
3,east,2016-03-10,34.0,15.0,8.0,5.0,0.60,38.0
4,central,2016-03-10,43.0,20.0,10.0,7.0,0.35,34.0
...,...,...,...,...,...,...,...,...
16860,west,2024-01-01,28.0,16.0,7.0,2.0,1.00,7.0
16861,east,2024-01-01,41.0,21.0,10.0,2.0,1.00,10.0
16862,central,2024-01-01,42.0,23.0,10.0,4.0,0.00,18.0
16863,south,2024-01-01,37.0,18.0,9.0,2.0,0.00,11.0


In [5]:
psi_df.region.unique()

array(['national', 'south', 'north', 'east', 'central', 'west'],
      dtype=object)

In [6]:
psi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16865 entries, 0 to 16864
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   region    16865 non-null  object 
 1   date      16865 non-null  object 
 2   psi_24hr  16865 non-null  float64
 3   pm10      16865 non-null  float64
 4   pm25      16865 non-null  float64
 5   so2       16865 non-null  float64
 6   co        16865 non-null  float64
 7   o3        16865 non-null  float64
dtypes: float64(6), object(2)
memory usage: 1.0+ MB


In [7]:
psi_df['date']= pd.to_datetime(psi_df['date'])
psi_df.date.describe()

C:\Users\dhind\AppData\Local\Temp/ipykernel_35452/201840204.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  psi_df.date.describe()


count                   16865
unique                   2821
top       2016-03-10 00:00:00
freq                        6
first     2016-03-10 00:00:00
last      2024-01-01 00:00:00
Name: date, dtype: object

In [8]:
## first     2016-03-10 00:00:00
## last      2024-01-01 00:00:00

## Vehicle Type Data 

In [11]:
import requests
import pandas as pd
from datetime import datetime, timedelta

class VehicleTypeData:
    def __init__(self, api_key):
        self.api_url = "http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation"
        self.api_key = api_key
        self.headers = {"AccountKey": api_key, "accept": "application/json"}
        
    def get_vehicle_distribution(self, start_date, end_date):
        """
        Fetch vehicle population by fuel type between a specific date range.
        
        Args:
            start_date (str): Start date in 'YYYY-MM-DD' format (e.g., '2014-01-01').
            end_date (str): End date in 'YYYY-MM-DD' format (e.g., '2024-01-01').
            
        Returns:
            pd.DataFrame: DataFrame containing vehicle distribution by fuel type over time.
        """
        vehicle_data = []
        
        # Convert string dates to datetime objects
        start = datetime.strptime(start_date, '%Y-%m-%d')
        end = datetime.strptime(end_date, '%Y-%m-%d')
        delta = timedelta(days=1)

        # Loop through each day in the range
        while start <= end:
            date_str = start.strftime('%Y-%m-%d')
            
            # Update your URL to include date parameter if API supports it
            # e.g., ?date=2014-01-01 or however the API expects the date format
            try:
                response = requests.get(self.api_url, headers=self.headers, params={"date": date_str})
                response.raise_for_status()
                data = response.json()

                for vehicle in data['value']:
                    vehicle_entry = {
                        "date": date_str,
                        "fuel_type": vehicle['FuelTypeDesc'],
                        "count": vehicle['VehicleCount']  # Assuming a field for count exists
                    }
                    vehicle_data.append(vehicle_entry)

            except requests.exceptions.RequestException as e:
                print(f"Error fetching vehicle data for {date_str}: {e}")

            # Move to the next day
            start += delta

        # Convert list of dictionaries to DataFrame
        vehicle_df = pd.DataFrame(vehicle_data)
        return vehicle_df

vehicle_api_key = "" 
# Example usage:
vehicle_data = VehicleTypeData(vehicle_api_key)
vehicle_distribution_df = vehicle_data.get_vehicle_distribution("2016-01-01", "2024-01-01")
print(vehicle_distribution_df.head())


Error fetching vehicle data for 2014-01-01: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-01-01
Error fetching vehicle data for 2014-01-02: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-01-02
Error fetching vehicle data for 2014-01-03: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-01-03
Error fetching vehicle data for 2014-01-04: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-01-04
Error fetching vehicle data for 2014-01-05: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-01-05
Error fetching vehicle data for 2014-01-06: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-01-06
Error fetching vehicle

Error fetching vehicle data for 2014-02-24: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-02-24
Error fetching vehicle data for 2014-02-25: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-02-25
Error fetching vehicle data for 2014-02-26: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-02-26
Error fetching vehicle data for 2014-02-27: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-02-27
Error fetching vehicle data for 2014-02-28: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-02-28
Error fetching vehicle data for 2014-03-01: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-03-01
Error fetching vehicle

Error fetching vehicle data for 2014-04-21: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-04-21
Error fetching vehicle data for 2014-04-22: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-04-22
Error fetching vehicle data for 2014-04-23: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-04-23
Error fetching vehicle data for 2014-04-24: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-04-24
Error fetching vehicle data for 2014-04-25: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-04-25
Error fetching vehicle data for 2014-04-26: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-04-26
Error fetching vehicle

Error fetching vehicle data for 2014-06-13: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-06-13
Error fetching vehicle data for 2014-06-14: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-06-14
Error fetching vehicle data for 2014-06-15: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-06-15
Error fetching vehicle data for 2014-06-16: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-06-16
Error fetching vehicle data for 2014-06-17: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-06-17
Error fetching vehicle data for 2014-06-18: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-06-18
Error fetching vehicle

Error fetching vehicle data for 2014-08-08: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-08-08
Error fetching vehicle data for 2014-08-09: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-08-09
Error fetching vehicle data for 2014-08-10: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-08-10
Error fetching vehicle data for 2014-08-11: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-08-11
Error fetching vehicle data for 2014-08-12: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-08-12
Error fetching vehicle data for 2014-08-13: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2014-08-13
Error fetching vehicle

...

Error fetching vehicle data for 2016-04-30: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-04-30
Error fetching vehicle data for 2016-05-01: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-05-01
Error fetching vehicle data for 2016-05-02: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-05-02
Error fetching vehicle data for 2016-05-03: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-05-03
Error fetching vehicle data for 2016-05-04: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-05-04
Error fetching vehicle data for 2016-05-05: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-05-05
Error fetching vehicle

Error fetching vehicle data for 2016-06-26: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-06-26
Error fetching vehicle data for 2016-06-27: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-06-27
Error fetching vehicle data for 2016-06-28: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-06-28
Error fetching vehicle data for 2016-06-29: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-06-29
Error fetching vehicle data for 2016-06-30: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-06-30
Error fetching vehicle data for 2016-07-01: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-07-01
Error fetching vehicle

Error fetching vehicle data for 2016-08-16: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-08-16
Error fetching vehicle data for 2016-08-17: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-08-17
Error fetching vehicle data for 2016-08-18: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-08-18
Error fetching vehicle data for 2016-08-19: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-08-19
Error fetching vehicle data for 2016-08-20: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-08-20
Error fetching vehicle data for 2016-08-21: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-08-21
Error fetching vehicle

Error fetching vehicle data for 2016-10-10: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-10-10
Error fetching vehicle data for 2016-10-11: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-10-11
Error fetching vehicle data for 2016-10-12: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-10-12
Error fetching vehicle data for 2016-10-13: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-10-13
Error fetching vehicle data for 2016-10-14: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-10-14
Error fetching vehicle data for 2016-10-15: 404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/VehiclePopulation?date=2016-10-15
Error fetching vehicle

KeyboardInterrupt: 

In [14]:
import requests
from datetime import datetime, timedelta
import time

class TrafficCongestionData:
    def __init__(self, api_key):
        self.api_url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
        self.api_key = api_key
    
    def get_traffic_data(self, coordinates, date):
        """Fetch traffic congestion data for given coordinates and date."""
        try:
            response = requests.get(
                f"{self.api_url}?point={coordinates}&unit=KMPH&key={self.api_key}"
            )
            response.raise_for_status()
            data = response.json()
            return data['flowSegmentData']
        except requests.exceptions.RequestException as e:
            print(f"Error fetching traffic data for {date}: {e}")
            return None

# Neighborhood coordinates for regions
neighborhood_coordinates = {
    "north": ["1.436,103.786", "1.449,103.818", "1.429,103.836"],  # Woodlands, Sembawang, Yishun
    "south": ["1.249,103.830", "1.290,103.859", "1.286,103.820"],  # Sentosa, Marina Bay, Bukit Merah
    "east": ["1.345,103.944", "1.321,103.927", "1.373,103.949"],   # Tampines, Bedok, Pasir Ris
    "west": ["1.332,103.743", "1.352,103.717", "1.356,103.762"],    # Jurong East, Jurong West, Bukit Batok
    "central": ["1.304,103.831", "1.332,103.847", "1.320,103.844"]  # Orchard, Toa Payoh, Novena
}

# Example function to estimate vehicle count from flow data
def estimate_vehicle_count(flow_data):
    """
    Estimate vehicle count based on road capacity, speed, and congestion level.
    This is a simplified approach, assuming 2000 vehicles/hour/lane capacity.
    """
    avg_speed = flow_data.get("currentSpeed", 0)
    avg_congestion = flow_data.get("currentTravelTime", 1) / flow_data.get("freeFlowTravelTime", 1)
    
    # Assume 1,800 vehicles per hour per lane
    estimated_flow = 1800 * avg_congestion * (avg_speed / 100)
    return estimated_flow

# Main function to extract daily traffic data for each neighborhood from 2016 to 2024
def extract_daily_traffic_data(api_key, start_date, end_date):
    traffic_data = TrafficCongestionData(api_key)
    current_date = start_date
    aggregated_traffic_data = []
    
    while current_date <= end_date:
        daily_data = {}
        print(f"Fetching traffic data for {current_date.strftime('%Y-%m-%d')}")
        
        for region, neighborhoods in neighborhood_coordinates.items():
            total_vehicle_count = 0
            for neighborhood in neighborhoods:
                flow_data = traffic_data.get_traffic_data(neighborhood, current_date)
                if flow_data:
                    vehicle_count = estimate_vehicle_count(flow_data)
                    total_vehicle_count += vehicle_count
            
            # Average vehicle count across neighborhoods in the region
            daily_data[region] = total_vehicle_count / len(neighborhoods)
        
        aggregated_traffic_data.append({
            "date": current_date.strftime('%Y-%m-%d'),
            "traffic_data": daily_data
        })
        
        # Go to the next day
        current_date += timedelta(days=1)
        time.sleep(1)  # To avoid hitting API rate limits

    return aggregated_traffic_data


# Define start and end dates
start_date = datetime(2016, 1, 1)
end_date = datetime(2016, 2, 1)

# Extract the traffic data
api_key = ""
traffic_data = extract_daily_traffic_data(api_key, start_date, end_date)

# Example of printing the aggregated traffic data
#for day_data in traffic_data:
#    print(f"Date: {day_data['date']}, Traffic Data: {day_data['traffic_data']}")

Fetching traffic data for 2016-01-01
Fetching traffic data for 2016-01-02
Fetching traffic data for 2016-01-03
Fetching traffic data for 2016-01-04
Fetching traffic data for 2016-01-05
Fetching traffic data for 2016-01-06
Fetching traffic data for 2016-01-07
Fetching traffic data for 2016-01-08
Fetching traffic data for 2016-01-09
Fetching traffic data for 2016-01-10
Fetching traffic data for 2016-01-11
Fetching traffic data for 2016-01-12
Fetching traffic data for 2016-01-13
Fetching traffic data for 2016-01-14
Fetching traffic data for 2016-01-15
Fetching traffic data for 2016-01-16
Fetching traffic data for 2016-01-17
Fetching traffic data for 2016-01-18
Fetching traffic data for 2016-01-19
Fetching traffic data for 2016-01-20
Fetching traffic data for 2016-01-21
Fetching traffic data for 2016-01-22
Fetching traffic data for 2016-01-23
Fetching traffic data for 2016-01-24
Fetching traffic data for 2016-01-25
Fetching traffic data for 2016-01-26
Fetching traffic data for 2016-01-27
F

In [15]:
traffic_data

[{'date': '2016-01-01',
  'traffic_data': {'north': 720.7067095580034,
   'south': 774.0696641313858,
   'east': 720.0931877930292,
   'west': 774.237790422001,
   'central': 743.9505837972298}},
 {'date': '2016-01-02',
  'traffic_data': {'north': 720.7067095580034,
   'south': 774.0696641313858,
   'east': 720.0931877930292,
   'west': 774.237790422001,
   'central': 743.9505837972298}},
 {'date': '2016-01-03',
  'traffic_data': {'north': 720.7067095580034,
   'south': 774.0094231675304,
   'east': 726.399866804647,
   'west': 750.3531881062172,
   'central': 750.6253713606653}},
 {'date': '2016-01-04',
  'traffic_data': {'north': 720.6826453868804,
   'south': 780.1993993164637,
   'east': 726.399866804647,
   'west': 750.3531881062172,
   'central': 750.6253713606653}},
 {'date': '2016-01-05',
  'traffic_data': {'north': 720.6826453868804,
   'south': 780.1993993164637,
   'east': 726.399866804647,
   'west': 750.3531881062172,
   'central': 750.6494355317885}},
 {'date': '2016-01-0